## Build Data

In [ ]:
filename = 'samples_kiss_pro.jsonl'
lang = 'English'

In [ ]:
def filename_suffix(fn, suffix):
    return fn.replace('.jsonl', f"_{suffix}.jsonl")
filename_suffix(filename, 'test')

In [ ]:
import jsonlines
samples_pro = []
with jsonlines.open(filename) as reader:
    for item in reader:
        samples_pro.append(item)

In [ ]:
samples_pro = [sample for sample in samples_pro if 'memory_maze' in sample]
# samples_pro.pop(334)
# samples_pro.pop(334)
# samples_pro.pop(391)
len(samples_pro)

In [ ]:
import sys
sys.path.append('..')
from agents.memory_tree import MemoryTree
def build_lines_one(sample, end_idx=10):
    mem = MemoryTree(sample['memory_maze'])
    sents = mem.get_originals()
    summary, _ = mem.get_summary(end_idx=end_idx)
    ret = summary + sents[end_idx:end_idx+55-len(summary)]
    return ret
build_lines_one(samples_pro[10], 200)

In [ ]:
def build_sys(j):
    sys = "Style: \n{0}\n\nBackground: \n{1}\n\nPlot: \n{2}\n\nProtagonists: \n{3}".format(j['style'], j['background'], j['plot'], j['protagonists'])
    sys2 = 'We are writing this story sentence by sentence. Please follow the setting and style. The outline should be finished in around 10k words. '
    return {'role':'system', 'content': sys+'\n\n'+sys2}
print(build_sys(samples_pro[0])['content'])

In [ ]:
def lines2turns(lines, sys, lang='Chinese'):
    user1 = f'Write the next sentence of the story in {lang}. '
    user2 = lines[0].strip()
    turns1 = [{'role': 'system', 'content': sys}]
    turns2 = [{'role': 'system', 'content': sys}]
    turns1 += [{'role': 'user', 'content': user1}]
    turns2 += [{'role': 'user', 'content': user1+user2}]
    for i in range(0, len(lines)-1, 2):
        turns1.append({'role': 'assistant', 'content': lines[i]})
        turns1.append({'role': 'user', 'content': lines[i+1]})
    for i in range(1, len(lines)-1, 2):
        turns2.append({'role': 'assistant', 'content': lines[i]})
        turns2.append({'role': 'user', 'content': lines[i+1]})
    return turns1, turns2
turns1, turns2 = lines2turns(build_lines_one(samples_pro[10], 200), build_sys(samples_pro[0])['content'], lang)
turns1

In [ ]:
messages_lst = []
import random
from tqdm import tqdm
i = 0
for sample in tqdm(samples_pro):
    i += 1
    mem = MemoryTree(sample['memory_maze'], lang=lang)
    end_idxs = random.sample(range(2, len(mem.get_originals())), 20)
    for end_idx in end_idxs:
        turns1, turns2 = lines2turns(build_lines_one(sample, end_idx), build_sys(sample)['content'], lang)
        messages_lst.append(turns1)
        messages_lst.append(turns2)

In [ ]:
print(len(messages_lst))
messages_lst[100]

In [ ]:
import json
def lst2jsonl(messages_lst, fn):
    with open(fn, "w") as f:
        for j in messages_lst:
            f.write(json.dumps({"messages": j}) + "\n")
lst2jsonl(messages_lst, filename_suffix(filename, 'mem'))

In [ ]:
def deduplicate_jsonl(input_file, output_file):
    # Create a set to store unique lines of the JSONL file
    unique_lines = set()

    # Open the input file in read mode
    with open(input_file, 'r') as f:
        # Read each line from the input file
        for line in f:
            # Convert the line to a dictionary
            data = json.loads(line.strip())
            # Convert the dictionary back to a string
            json_string = json.dumps(data, sort_keys=True)
            # Add the string to the set
            unique_lines.add(json_string)

    # Open the output file in write mode
    with open(output_file, 'w') as f:
        # Write each unique line to the output file
        for line in unique_lines:
            f.write(line + '\n')

# Example usage
deduplicate_jsonl(filename_suffix(filename, 'mem'), filename_suffix(filename_suffix(filename, 'mem'), 'deduplicated'))

In [ ]:
import random
def random_sample_half(input_file, output_file):
    # Read lines from input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Get half the length of lines
    half_length = len(lines) // 2

    # Randomly select half the lines
    sampled_lines = random.sample(lines, half_length)

    print(len(lines), len(sampled_lines))

    # Save sampled lines to output file
    with open(output_file, 'w') as outfile:
        outfile.writelines(sampled_lines)
random_sample_half(filename_suffix(filename_suffix(filename, 'mem'), 'deduplicated'), filename_suffix(filename_suffix(filename_suffix(filename, 'mem'), 'deduplicated'), 'half'))